In [1]:
!pip install transformers
!pip install datasets
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.9 MB/s eta 0:00:0

In [2]:
import pandas as pd
import re
import io
import json
import os
import ast

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

import transformers
from transformers import AdamW, get_scheduler, GPTNeoModel, GPT2LMHeadModel
import datetime

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

from google.colab import drive
drive.mount('/content/drive')
import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'deeplearning/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

models_folder="trained_models"

Mounted at /content/drive
['shortjokes.csv', 'openseminar_GAN.pptm', 'openseminar_GAN.pptm.gslides', 'Untitled0.ipynb', 'openseminar_GAN_final.docx', 'Davinci003.ipynb', 'Untitled', 'GPTNEO.ipynb', 'baseline.ipynb', 'RM_Training.ipynb', 'trained_models', 'Transformers.ipynb', 'QA_RL_PPO.ipynb', 'api.ipynb', 'RLHF_Dataset (1).ipynb', 'GPTNeoMyself.ipynb', 'InstructMyself2.ipynb', 'q-learning.ipynb', 'deepdai', 'VAE.ipynb', 'diffusion', 'competition', 'GPTNeo_RLHF.ipynb', 'CRS', 'reward_logs.txt']


In [3]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M", bos_token='<|startoftext|>', eos_token='<|endoftext|>')
tokenizer.pad_token = tokenizer.eos_token

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

# get fine-tuned model?
model_path = GOOGLE_DRIVE_PATH + models_folder + f"/gpt_ing.pt"
model.load_state_dict(torch.load(model_path)) # 여기서는 dict만 불러온다. 애초에 dict만 저장함.

model.to(device)
model.eval()
print("skip")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


skip


In [21]:
prompt=f"Human:Transform the following sentence into the passive voice.\n\nInformation:I bought a book\n\nAssistant:"
tokenized = tokenizer(prompt, return_tensors="pt")
input_ids, attention_mask = tokenized['input_ids'].to(device), tokenized['attention_mask'].to(device)

generated = model.generate(
    input_ids, attention_mask=attention_mask, max_length=1024, num_beams=5, no_repeat_ngram_size=3, early_stopping=True, temperature=0.3,
)
print(tokenizer.decode(generated[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Human:Transform the following sentence into the passive voice.

Information:I bought a book

Assistant:I buy a book.


In [ ]:
def harry_chat(input):
  # sim 계산
  # tf-idf 계산
  # 합쳐
  informations=""
  informations += json_data[0]
  found=find_informations(input)
  for x in list(found[0].values())[:2]:
    print(x)
    if x[0]>75:
      informations += x[1]
  
  # model에 넣어서 response 생성
  prompt=f"Human:{input} Use below information.\n\nInformation:{informations}\n\nAssistant:"
  tokenized = tokenizer(prompt, return_tensors="pt")
  input_ids, attention_mask = tokenized['input_ids'].to(device), tokenized['attention_mask'].to(device)

  generated = model.generate(
      input_ids, max_length=1024, num_beams=5, no_repeat_ngram_size=3, early_stopping=True, temperature=0.3,
  )
  return tokenizer.decode(generated[0], skip_special_tokens=True)

In [ ]:
harry_chat("What sports do you like the most?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1
=-------------------=-------------------=-------------------=-------------------
겹침 77.85 47.57 I was thrilled when Teddy, who had become like family to me, started dating Victoire Weasley and potentially becoming a part of our family.
겹침 75.68 35.3 Despite needing glasses, I have excellent eyesight and can locate even the smallest objects like the Golden Snitch on a Quidditch field.
[89.75147707607894, 'I was thrilled when Teddy, who had become like family to me, started dating Victoire Weasley and potentially becoming a part of our family.']
[84.50697978661876, 'Despite needing glasses, I have excellent eyesight and can locate even the smallest objects like the Golden Snitch on a Quidditch field.']


'Human:What sports do you like the most? Use below information.\n\nInformation:I am Harry Potter, and I was born to Lily and James Potter on July 31st, 1980.I was thrilled when Teddy, who had become like family to me, started dating Victoire Weasley and potentially becoming a part of our family.Despite needing glasses, I have excellent eyesight and can locate even the smallest objects like the Golden Snitch on a Quidditch field.\n\nAssistant: Harry Potter and the Deathly Hallows'

In [ ]:
def change_you_to_Harry(query):
  if 'your' in query:
    query=query.replace('your', "My")
  if 'you' in query:
    query=query.replace('you', 'I')
  query.replace('are', 'am')

  return query

In [ ]:
from datasets import Dataset
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import math
import collections

torch.set_grad_enabled(False) # https://runebook.dev/ko/docs/pytorch/generated/torch.set_grad_enabled
bert_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
bert_model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [ ]:
with open(GOOGLE_DRIVE_PATH+'deepdai/harry_data2.json') as json_file:
    json_data = json.load(json_file)

HarryDict = dict()
HarryDict['id']=[x for x in range(len(json_data))]
HarryDict['sent']=[x for x in json_data]

dataset = Dataset.from_dict(HarryDict)

embeddings1 = dataset.map(lambda x: {'embeddings': bert_model(**bert_tokenizer(x["sent"],padding=True, truncation=True, return_tensors='pt'))[0][0].numpy()})
embeddings1.add_faiss_index(column='embeddings')

Map:   0%|          | 0/589 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'sent', 'embeddings'],
    num_rows: 589
})

In [ ]:
num_of_sentences=30

def find_informations(query):
  def get_tf_results(queries):
    queries = [change_you_to_Harry(x) for x in queries]
    tf_weight_list = [x for x in range(50, 0, -1)]
    data = queries + json_data

    tfidf_vect = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vect.fit_transform(data)
    cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    tf_result = []
    for idx in range(len(queries)):
      injected_list=[]
      sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
      sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
      sim_scores = sim_scores[:num_of_sentences]
      i=0
      for (index, score) in sim_scores:
        if sim_scores[i][1]==0:
          cal_score=0
        else:
          cal_score=(sim_scores[i][1]/3+1.2)*tf_weight_list[i]
        injected_list.append({"index":index-len(queries), "score":cal_score, "sent":data[int(sim_scores[i][0])]})
        i+=1
      # for i in range(num_of_sentences):
      #     print(f'score : {injected_list[i]["score"]}, Found : {injected_list[i]["sent"]}')
      tf_result.append(injected_list)
    return tf_result

  def get_sim_results(queries):
    sim_results=[]
    k=num_of_sentences

    for orig_query in queries:
      query = change_you_to_Harry(orig_query)
      question_embedding = q_encoder(**q_tokenizer(query, return_tensors="pt"))[0][0].numpy()
      scores, retrieved_examples = embeddings1.get_nearest_examples('embeddings', question_embedding, k=k)
      df_dpr = pd.DataFrame.from_dict(retrieved_examples)
      df_dpr["scores"] = scores
      df_dpr.sort_values("scores", ascending=False, inplace=True)
      sim_results.append({
          "query":orig_query,
          "result":df_dpr[['id', 'sent', 'scores']]
      })
    return sim_results
  
  queries=[query]
  sim_results = get_sim_results(queries)
  queries = [preprocessing_query(x) for x in queries]
  tf_results=get_tf_results(queries)

  all_results=[]
  hyper = 0.25

  for (idx, sim) in enumerate(sim_results):
    print("=-------------------"*4)
    result_dict=collections.defaultdict(int)
    for i in range(len(sim['result'])):
      result_dict[sim['result'].iloc[i]['id']]=[sim['result'].iloc[i]['scores']*(1-hyper), sim['result'].iloc[i]['sent']]
    for tf in tf_results[idx]:
      if result_dict[tf['index']]:
        print("겹침", math.floor(result_dict[tf['index']][0]*100)/100, math.floor(tf['score']*100)/100, tf['sent'])
        result_dict[tf['index']] = [tf['score']*hyper + result_dict[tf['index']][0], result_dict[tf['index']][1]]
      else:
        result_dict[tf['index']]=[tf['score']*(hyper), tf['sent']]
    sorted_dict = dict(sorted(result_dict.items(), key=lambda item: item[1],reverse=True))
    all_results.append(sorted_dict)
  
  return all_results

In [ ]:
def preprocessing_query(data, plus=True):
  if "yours" in data:
    data=data.replace('yours', 'My')
  if 'your' in data:
    data=data.replace('your', 'My')
  if 'you' in data:
    print(1)
    data=data.replace('you', 'I')

  if plus == True:
    if "friend" in data:
      data += " friends"
    if "sport" in data:
      data += " sports"
  
  data=data.replace('are', 'am')
  
  return data

In [ ]:
def inference(prompt, info):
  prompt = "Human:"+prompt+"\n\nInformation:"+info+"\n\nAssistant:"
  tokenized = tokenizer(prompt, return_tensors="pt")

  input_ids, attention_mask = tokenized['input_ids'].to(device), tokenized['attention_mask'].to(device)

  generated = model.generate(
      input_ids, max_length=1024, num_beams=5, no_repeat_ngram_size=3, early_stopping=True, temperature=0.3,
  )

  return tokenizer.decode(generated[0], skip_special_tokens=True)

In [ ]:
inference("Hello! Say long sentence.", "")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'human : hello! say long sentence. information : assistant : [UNK]'